In [1]:
# here is function that perfoms ABC test

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('data.txt', encoding='windows-1251')

In [3]:
df.head(1)

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,...,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-11,10:15:35,2173,2004598,2,22577,Розничная реализация,18,45399,ЦИПРОЛЕТ 3МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /Д-...,...,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.0,41.08,51.0,12.0,925.0,2.000000e+11,205,1,1.0


In [4]:
df_exmpl = df[['DR_NDrugs', 'DR_Kol', 'DR_CRoz']]
df_exmpl

,DR_NDrugs,DR_Kol,DR_CRoz
0,ЦИПРОЛЕТ 3МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /Д-...,1.0,51.0
1,ПЕРЕКИСЬ ВОДОРОДА 3% 100МЛ. №40 Р-Р ФЛ.,1.0,31.0
2,СОФЬЯ ГЕЛЬ Д/НОГ ВЕНОТОНИЗ. ТРОКСЕРУТИН ФОРТЕ ...,1.0,209.0
3,СОФЬЯ ГХК КРЕМ Д/ТЕЛА ХОНДРОИТИН+ГЛЮКОЗАМИН 12...,1.0,210.0
4,ГАЛВУС 50МГ. №28 ТАБ. /НОВАРТИС/,1.0,787.0
...,...,...,...
4457,КЕТОРОЛ ЭКСПРЕСС 10МГ. №20 ТАБ. ДИСПЕРГ. /Д-Р ...,1.0,75.0
4458,АНТИПОЛИЦАЙ ВАЙТ №6 ТАБ.,1.0,93.0
4459,СИЛДЕНАФИЛ-СЗ 50МГ. №10 ТАБ. П/П/О /СЕВЕРНАЯ З...,1.0,396.0
4460,ТОБРАЗОН 5МЛ. ГЛ.КАПЛИ ФЛ.,1.0,419.0


In [5]:
a = list(df_exmpl.columns)
a.remove('DR_Kol')
a

['DR_NDrugs', 'DR_CRoz']

In [ ]:
def perform_abc(df, index):
    '''
    dataframe argument is the source of data, index is the grouping argument
    sourse data should be cleaned and only have columns that are needed to ABC analysis
    '''
    sum_columns = list(df.columns)
    sum_columns.remove(index)

    result_df = df.groupby(index).agg({column: 'sum' for column in sum_columns}) # here im creating dicrionary with rules of agg for columns
    # here im doing in loop what i did in manual abc tests for each column in sum_columns
    for column in sum_columns:
        result_df[f'{column}_share'] = result_df[column] / sum(result_df[column])
        result_df = result_df.sort_values(f'{column}_share', ascending=False)
        result_df[f'{column}_cumsum'] = result_df[f'{column}_share'].cumsum()
        result_df[f'{column}_abc'] = np.where(result_df[f'{column}_cumsum'] < 0.8, 'A', np.where(result_df[f'{column}_cumsum'] < 0.95, 'B', 'C'))
    return result_df[sum_columns + [f'{column}_abc' for column in sum_columns]]

In [11]:
perform_abc(df=df_exmpl, index='DR_NDrugs')

,DR_Kol,DR_CRoz,DR_Kol_abc,DR_CRoz_abc
DR_NDrugs,,,,
"ТЕРАФЛЮ ЛИМОН ОТ ГРИППА И ПРОСТУДЫ 22,1Г. №14 ПОР. Д/Р-РА Д/ПРИЕМА ВНУТРЬ ПАК.",7.000000,15638.00,A,A
НИМЕСИЛ 100МГ. 2Г. №30 ГРАН. Д/СУСП. Д/ПРИЕМА ВНУТРЬ ПАК. /ГУИДОТТИ/МЕНАРИНИ/,2.800000,14445.00,A,A
ИНГАВИРИН 90МГ. №10 КАПС. /ВАЛЕНТА/,18.000000,13750.00,A,A
ХАРТМАНН БРАНОЛИНД H ПОВЯЗКА СТЕР. 10Х20СМ. №30 ПЕРУАН.БАЛЬЗАМ /АРТ.4923462/ [BRANOLIND],0.166667,12908.00,C,A
"ЭЛИКВИС 2,5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/",5.000000,12731.00,A,A
...,...,...,...,...
КОНТЕЙНЕР Д/СБОРА БИОМАТЕРИАЛА 60МЛ. +ШПАТЕЛЬ СТЕР. И/У (БАНКА),1.000000,10.00,C,C
АСКОРБИНОВАЯ К-ТА 25МГ. АПЕЛЬСИН №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/,1.000000,10.00,B,C
АСКОРБИНОВАЯ К-ТА 25МГ. №10 ТАБ. КРУТКА САХ. /АСКОПРОМ/,7.000000,9.00,A,C
